# SQLAlchemy ORM Schema Design
Table schema example using postgresql

In [ ]:
from datetime import datetime, date
import json
from os import environ

from sqlalchemy import create_engine, text, func
from sqlalchemy.exc import SQLAlchemyError
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, aliased, selectinload, joinedload
from sqlalchemy import Column, ForeignKey, Integer, String, Float
from sqlalchemy import BigInteger, Boolean, Date, DateTime, Text
from sqlalchemy.orm import relationship, backref
from sqlalchemy.orm.exc import MultipleResultsFound
from sqlalchemy.sql import exists

# Setup

In [ ]:
host = environ.get('PG_SERVER', 'sql-fabulous')
db = environ.get('PG_DATABASE', 'condesa')
user = environ.get('PG_UID', 'sa')
pw = environ.get('POSTGRES_PASSWORD', 'pwd')

con_str = f'{user}:{pw}@{host}/{db}'

# 'echo' emits generated sql
engine = create_engine(f"postgresql+psycopg2://{con_str}", echo=True)

# Define the catalog of classes

In [ ]:
Base = declarative_base()


class PracticeInterface(Base):
    __tablename__ = 'practice_interface'
    
    id = Column(BigInteger, autoincrement=True, primary_key=True)
    
    data_partner_registry_id = Column(BigInteger, ForeignKey('data_partner_registry.id'), nullable=False)
    practice_interface_vendor_id = Column(BigInteger, ForeignKey('practice_interface_vendor.id'), nullable=False)
    practice_interface_category_id = Column(BigInteger, ForeignKey('practice_interface_category.id'), nullable=False)
    practice_interface_pipeline_id = Column(BigInteger, ForeignKey('practice_interface_pipeline.id'), nullable=False)
    practice_interface_status_id = Column(BigInteger, ForeignKey('practice_interface_status.id'), nullable=False)

    prac_id = Column(BigInteger, nullable=False)
    is_inbound = Column(Boolean, default=True, nullable=False)
    
    start_date = Column(Date, default=date.today, nullable=False)
    end_date = Column(Date, default=None, nullable=True)
    
    created_at = Column(DateTime, default=datetime.now, nullable=True)
    updated_at = Column(DateTime, default=datetime.now, onupdate=datetime.now, nullable=True)

    data_partner_registry = relationship('DataPartnerRegistry', backref=backref('interfaces', uselist=True))
    practice_interface_vendor = relationship('PracticeInterfaceVendor', backref=backref('vendors', uselist=True))
    practice_interface_category = relationship('PracticeInterfaceCategory', backref=backref('categories', uselist=True))
    practice_interface_pipeline = relationship('PracticeInterfacePipeline', backref=backref('pipelines', uselist=True))
    practice_interface_status = relationship('PracticeInterfaceStatus', backref=backref('statuses', uselist=True))

    def __repr__(self):
        return ("PracticeInterface(id={}, source={}, vendor={}, category={}, status={})".
               format(self.id, self.data_partner_registry, 
                      self.practice_interface_vendor, self.practice_interface_category,
                      self.practice_interface_status))

class DataPartnerRegistry(Base):
    __tablename__ = 'data_partner_registry'
    
    id = Column(BigInteger, autoincrement=True, primary_key=True)
    name = Column(Text, nullable=False, unique=True)
    description = Column(Text, nullable=True)

    created_at = Column(DateTime, default=datetime.now, nullable=True)
    updated_at = Column(DateTime, default=datetime.now, onupdate=datetime.now, nullable=True)

    def __repr__(self):
        return "Registry(id={0}, name={1})".format(self.id, self.name)

class PracticeInterfaceVendor(Base):
    __tablename__ = 'practice_interface_vendor'
    
    id = Column(BigInteger, autoincrement=True, primary_key=True)
    name = Column(Text, nullable=False, unique=True)
    description = Column(Text, nullable=True)

    created_at = Column(DateTime, default=datetime.now, nullable=True)
    updated_at = Column(DateTime, default=datetime.now, onupdate=datetime.now, nullable=True)

    def __repr__(self):
        return "Vendor(id={0}, name={1})".format(self.id, self.name)

class PracticeInterfaceCategory(Base):
    __tablename__ = 'practice_interface_category'
    
    id = Column(BigInteger, autoincrement=True, primary_key=True)
    name = Column(Text, nullable=False, unique=True)
    description = Column(Text, nullable=True)

    created_at = Column(DateTime, default=datetime.now, nullable=True)
    updated_at = Column(DateTime, default=datetime.now, onupdate=datetime.now, nullable=True)

    def __repr__(self):
        return "Category(id={0}, name={1})".format(self.id, self.name)

class PracticeInterfacePipeline(Base):
    __tablename__ = 'practice_interface_pipeline'
    
    id = Column(BigInteger, autoincrement=True, primary_key=True)
    name = Column(Text, nullable=False, unique=True)
    description = Column(Text, nullable=True)

    created_at = Column(DateTime, default=datetime.now, nullable=True)
    updated_at = Column(DateTime, default=datetime.now, onupdate=datetime.now, nullable=True)

    def __repr__(self):
        return "Pipeline(id={0}, name={1})".format(self.id, self.name)

class PracticeInterfaceStatus(Base):
    __tablename__ = 'practice_interface_status'
    
    id = Column(BigInteger, autoincrement=True, primary_key=True)
    name = Column(Text, nullable=False, unique=True)
    description = Column(Text, nullable=True)

    created_at = Column(DateTime, default=datetime.now, nullable=True)
    updated_at = Column(DateTime, default=datetime.now, onupdate=datetime.now, nullable=True)

    def __repr__(self):
        return "Status(id={0}, name={1})".format(self.id, self.name)
    

# For SQL Core: set mapped tables to local vars so that full SQL metadata is available.
interfaces = PracticeInterface.__table__
vendors = PracticeInterfaceVendor.__table__
categories = PracticeInterfaceCategory.__table__
pipelines = PracticeInterfacePipeline.__table__
statuses = PracticeInterfaceStatus.__table__

# Create session - interface to the DB

In [ ]:
Session = sessionmaker(bind=engine)
session = Session()

# Create Schema

In [ ]:
Base.metadata.drop_all(engine, checkfirst=True)
Base.metadata.create_all(engine, checkfirst=False)

# Inspect underlying metadata

In [ ]:
DataPartnerRegistry.__table__

# Create session - interface to the DB

In [ ]:
Session = sessionmaker(bind=engine)
session = Session()

# Create Beans

### Reference Data

In [ ]:
try:
    # Automatically creates a constructor accepting kwargs
    athena = DataPartnerRegistry(name='athena')
    
    session.add(athena)
    
    athena_vendor = PracticeInterfaceVendor(name='athena')
    ecw = PracticeInterfaceVendor(name='ecw')
    healthjump = PracticeInterfaceVendor(name='healthjump')

    session.add_all([athena_vendor, ecw, healthjump])

    unknown = PracticeInterfaceCategory(name='unknown')
    clinical = PracticeInterfaceCategory(name='clinical')
    qdc = PracticeInterfaceCategory(name='qdc')
    appointment = PracticeInterfaceCategory(name='appointment')
    hie = PracticeInterfaceCategory(name='hie')
    labs = PracticeInterfaceCategory(name='labs')
    claims = PracticeInterfaceCategory(name='claims')

    session.add_all([unknown, clinical, qdc, appointment, hie, labs, claims])
    
    mc_ccda = PracticeInterfacePipeline(name='MAeHC CCDA', description='MAeHC CCDA practice interface')
    cd_ccda = PracticeInterfacePipeline(name='Aledade CCDA', description='Aledade clinical_quality_ccd data pipeline')

    session.add_all([mc_ccda, cd_ccda])
    
    active = PracticeInterfaceStatus(name='active', description='MAeHC CCDA practice interface')
    inactive = PracticeInterfaceStatus(name='inactive', description='MAeHC CCDA practice interface')
    terminated = PracticeInterfaceStatus(name='terminated', description='MAeHC CCDA practice interface')
    paused = PracticeInterfaceStatus(name='paused', description='MAeHC CCDA practice interface')

    session.add_all([active, inactive, terminated, paused])
  
    session.commit()

except SQLAlchemyError as e:
    session.rollback()
    raise

### Interfaces

In [ ]:
try:
    athena = (session.
              query(DataPartnerRegistry).
              filter(DataPartnerRegistry.name == 'athena').
              one())
    
    athena_vendor = (session.
                     query(PracticeInterfaceVendor).
                     filter(PracticeInterfaceVendor.name == 'athena').
                     one())
    clinical = (session.
                query(PracticeInterfaceCategory).
                filter(PracticeInterfaceCategory.name == 'clinical').
                one())

    mc_ccda = (session.
               query(PracticeInterfacePipeline).
               filter(PracticeInterfacePipeline.name == 'MAeHC CCDA').
               one())
    
    active = (session.
              query(PracticeInterfaceStatus).
              filter(PracticeInterfaceStatus.name == 'active').
              one())
    
    interface_1 = PracticeInterface(data_partner_registry=athena, 
                                    practice_interface_vendor=athena_vendor,
                                    practice_interface_category=clinical,
                                    practice_interface_pipeline=mc_ccda,
                                    practice_interface_status=active, 
                                    prac_id=1,
                                   )

    session.add(interface_1)
    
    session.commit()

except SQLAlchemyError as e:
    session.rollback()
    raise